In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

% matplotlib inline

In [ ]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

training_data = [
    ("Doug saw Judy".split(), [0,1,2], "saw Judy .".split(), [1,2,3]),
    ("saw Judy .".split(), [1,2,3], "Judy . Judy".split(), [2,3,2]),
    ("Judy . Judy".split(), [2,3,2], ". Judy saw".split(), [3,2,1]),
    (". Judy saw".split(), [3,2,1], "Judy saw Brad".split(), [2,1,4]),
    ("Judy saw Brad".split(), [2,1,4], "saw Brad .".split(), [1,4,3]),
    ("saw Brad .".split(), [1,4,3], "Brad . Brad".split(), [4,3,4]),
    ("Brad . Brad".split(), [4,3,4], ". Brad saw".split(), [3,4,1]),
    (". Brad saw".split(), [3,4,1], "Brad saw Doug".split(), [4,1,0]),
    ("Brad saw Doug".split(), [4,1,0], "saw Doug .".split(), [1,0,3]),
    ("saw Doug .".split(), [1,0,3], "Doug . Doug".split(), [0,3,0]),
    ("Doug . Doug".split(), [0,3,0], ". Doug saw".split(), [3,0,1]),
    (". Doug saw".split(), [3,0,1], "Doug saw Judy".split(), [0,1,2])
]

word_to_ix = {}
for inp, t_in, out, t_out in training_data:
    for word in inp:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {'Doug': 0, 'saw': 1, 'Judy': 2, '.': 3, 'Brad': 4}
output_dict = {0:'Doug', 1:'saw', 2:'Judy', 3:'.', 4:'Brad'}

In [ ]:
INPUT_SIZE = 6
OUTPUT_SIZE = 6
LAYER_SIZE = 1
BATCH_SIZE = 1

class CBLSTM(nn.Module):

    def __init__(self, INPUT_SIZE, OUTPUT_SIZE, vocab_size, tagset_size):
        super(CBLSTM, self).__init__()
        self.hidden_dim = OUTPUT_SIZE
        self.vocab_size = vocab_size
        self.word_embeddings = nn.Embedding(vocab_size, INPUT_SIZE)
        
        self.lstm = nn.LSTM(INPUT_SIZE, OUTPUT_SIZE, LAYER_SIZE)

        self.hidden2tag = nn.Linear(OUTPUT_SIZE, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (autograd.Variable(torch.zeros(LAYER_SIZE, BATCH_SIZE, OUTPUT_SIZE)),
                autograd.Variable(torch.zeros(LAYER_SIZE, BATCH_SIZE, OUTPUT_SIZE)))

    def init_rand_hidden(self):
        return (autograd.Variable(torch.rand(LAYER_SIZE, BATCH_SIZE, OUTPUT_SIZE))*5,
                autograd.Variable(torch.rand(LAYER_SIZE, BATCH_SIZE, OUTPUT_SIZE))*5)
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence) # embed the sentence e.g. [0,1,2] to 3x6 (3 words, 6 embed size)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
model = CBLSTM(INPUT_SIZE, OUTPUT_SIZE, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=1)
loss_save = []
for epoch in range(300):
    for sentence, _, tags, _ in training_data:
        model.zero_grad()
        model.hidden = model.init_hidden()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        tag_scores = model(sentence_in)
        loss = loss_function(tag_scores, targets)
        loss_save.append(loss.data[0])
        loss.backward()
        optimizer.step()

plt.figure(figsize=[20,6])
plt.plot(loss_save)
plt.show()


# Input = 3 words, usually output perfect output

In [ ]:
childrenbook = []
inputs = prepare_sequence(training_data[9][0], word_to_ix)
for i in range(100):
    tag_scores = model(inputs)
    _, out = tag_scores[-1:,:].max(dim = 1)
    childrenbook.append(output_dict[int(out.data.numpy())])
    inputs = inputs[1:]
    nextin = autograd.Variable(torch.LongTensor([tag_to_ix[output_dict[int(out.data.numpy())]]]))
    inputs = torch.cat((inputs, nextin))

for i in range(int(len(childrenbook))):
    print(childrenbook[i], end = " ")

# Input = 1 word, more general

In [ ]:
childrenbook = []
inp = 0
inputs = autograd.Variable(torch.LongTensor([inp]))
print(output_dict[inp], end = " ")
for i in range(100):
    tag_scores = model(inputs)
    _, out = tag_scores[-1:,:].max(dim = 1)
    childrenbook.append(output_dict[int(out.data.numpy())])
    inputs = autograd.Variable(torch.LongTensor([tag_to_ix[output_dict[int(out.data.numpy())]]]))

for i in range(int(len(childrenbook))):
    print(childrenbook[i], end = " ")